In [1]:
from dash import Dash,html,dcc
import plotly.express as px
import pandas as pd
import numpy as np
from dash.dependencies import Input,Output,State
import dash_bootstrap_components as dbc

# insta_general

In [2]:
df = pd.read_csv('insta_general.csv', header=0)

In [3]:
df['posts']= df['posts'].astype('string').str.replace(',','').astype('int')

In [4]:
df['followers']= df['followers'].astype('int')
df['followings']= df['followings'].astype('int')

In [5]:
df0 = df.sort_values(by=['followers'])
followers_Bar = px.bar(df0 , y= 'name', x='followers', orientation='h')

In [6]:
df0 = df.sort_values(by=['posts'])
postsNum_Bar = px.bar(df0 , x= 'name', y='posts')

In [7]:
categories_followers_pie = px.pie(df, names='category', values='followers', title='Most Followed Category')

In [8]:
tier_pie = px.pie(df, names='tier', title='Tiers')

In [9]:
df0 = df.sort_values(by=['avg_engagement_rate'])
er_Bar = px.bar(df0 , x= 'username', y='avg_engagement_rate')

# insta_detailed

In [10]:
df_detailed = pd.read_csv('outputs/insta_detailed.csv', header=0)

In [11]:
df_temp = df_detailed.drop(df_detailed[df_detailed['likes']=='unknown'].index)
df_detailed['likes'] = df_detailed['likes'].replace('unknown',df_temp['likes'].astype('int').min())
df_detailed['likes'] = df_detailed['likes'].astype('int')
df_detailed['captions']=df_detailed['captions'].fillna(' ')

In [12]:
posts_by_type_pie = px.pie(df_detailed, names='type', title='Types of Posts')

In [13]:
df_detailed['datetimes'] = pd.to_datetime(df_detailed['datetimes'])

In [14]:
Likes_scatter = px.scatter(df_detailed,
                           x='datetimes',y='likes',
                           color='username',
                           hover_name='username',
                           width=1000,height=1000,
                           title='Date and Time of Post VS Likes',
                           symbol='username',
                           template='seaborn')

# Dash App

In [ ]:
app = Dash()
@app.callback(
    Output(component_id='graph1',component_property='figure'),
    Output(component_id='graph2',component_property='figure'),
    Output(component_id='graph3',component_property='figure'),
    Input(component_id='ddown1',component_property='value'),
    Input(component_id='ddown2',component_property='value'),
)
def update_My_Div(category,tier):
    if category==None and tier==None:
        df0 = df.sort_values(by=['followers'] , ascending=False)
        df1 = df.sort_values(by=['posts'], ascending=False)
        df2 = df.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar
    elif tier==None:
        df0 = df[df['category'] == category]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar
    elif category==None:
        df0 = df[df['tier'] == tier]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar,er_Bar
        
    else:
        df0 = df[np.logical_and(df['category'] == category,df['tier'] == tier)]
        df0 = df0.sort_values(by=['followers'], ascending=False)
        df1 = df0.sort_values(by=['posts'], ascending=False)
        df2 = df0.sort_values(by=['avg_engagement_rate'], ascending=False)
        followers_Bar = px.bar(df0 , x= 'name', y='followers')
        postsNum_Bar = px.bar(df1 , x= 'name', y='posts')
        er_Bar = px.bar(df2 , x= 'username', y='avg_engagement_rate')
        return followers_Bar,postsNum_Bar, er_Bar

@app.callback(
    Output(component_id='graph4',component_property='figure'),
    Input(component_id='ddown3',component_property='value'),
)
def update_My_Div(username):
    if username==None or len(username)==0:
        Likes_scatter = px.scatter(df_detailed,
                        x='datetimes',y='likes',
                        color='username',
                        hover_name='username',
                        width=1000,height=1000,
                        title='Date_and_Time_of_Post VS Likes',
                        symbol='username',
                        template='seaborn')
        return Likes_scatter
    else:
        df_scatter = df_detailed[np.isin(df_detailed.username,username)]
        Likes_scatter = px.scatter(df_scatter,
                        x='datetimes',y='likes',
                        color='username',
                        hover_name='username',
                        width=1000,height=1000,
                        title='Date_and_Time_of_Post VS Likes',
                        symbol='username',
                        template='seaborn')
        return Likes_scatter

@app.callback(
    Output(component_id='ddown5',component_property='options'),
    Input(component_id='ddown4',component_property='value'),
)

def update_My_Div(username):
    df_name = df_detailed[df_detailed['username']==username]
    options=[{'label': str(datetimes), 'value': str(datetimes)}  for datetimes in df_name['datetimes'].unique()]

    return options

@app.callback(
    Output(component_id='card1',component_property='children'),
    Input(component_id='ddown4',component_property='value'),
    Input(component_id='ddown5',component_property='value'),
)

def update_My_Div(username, datetimes):
    
    df_name = df_detailed[df_detailed['username']==username]
    
    if username == None:
        children = [dbc.CardImg(src="https://raw.githubusercontent.com/ayahhhany/Instagram-Dashboard/main/avatars/insta.png", top=True),]
    elif datetimes == None or ((df_name['datetimes']!=datetimes).all()) :
        children = [dbc.CardImg(src = df[df['username'] == username].iloc[0,8], top=True),
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                   ]
    else:
        children = [dbc.CardImg(src = df[df['username'] == username].iloc[0,8], top=True),
                    dbc.CardBody([html.H4(username, className="card-title"),]),
                    html.P(df_name[df_name['datetimes'] == datetimes].iloc[0,5], className="card-text",),
                    dbc.Button("Open Post", color="primary", href=df_name[df_name['datetimes'] == datetimes].iloc[0,1],),
                   ]

    return children

    
app.layout=html.Div(children=[
    
     html.Div(children=[
         html.Div('Combined Followers',style={'width':500,'fontSize' :40,}),
         html.Div('Total Posts',style={'width': 300, 'fontSize' :40,}),
         html.Div('Total Likes',style={'width': 300,'fontSize' :40,}),
         html.Div('Higher Engagement Rate',style={'width': 400,'fontSize' :40,}),
      ],style={'width': 1500,
              'display': 'flex',
              'flex-direction': 'row',
              'justify-content': 'center',}
        ),
     html.Div(children=[
         html.Div(str(df['followers'].sum()),style={'width':500,'fontSize' :40,}),
         html.Div(str(df['posts'].sum()),style={'width': 300, 'fontSize' :40,}),
         html.Div(str(df_detailed['likes'].sum()),style={'width': 300,'fontSize' :40,}),
         html.Div(str(round(df['avg_engagement_rate'].max(),2)),style={'width': 300, 'fontSize' :40,}),
      ],style={'width': 1500,
              'display': 'flex',
              'flex-direction': 'row',
              'justify-content': 'center',}
        ),    
    
     html.Div(dcc.Graph(figure=categories_followers_pie)),
     html.Div(dcc.Graph(figure=tier_pie)),
     html.Div(dcc.Graph(figure=posts_by_type_pie)),
    
    
     html.Div(dcc.Graph(id='graph4')),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)}  for username in df_detailed['username'].unique()],
                  id='ddown3',multi=True,
    ),
    
    
     dbc.Card(id='card1'),
     dcc.Dropdown(options=[{'label': str(username), 'value': str(username)}  for username in df['username'].unique()],
                      id='ddown4',
        ),
     dcc.Dropdown(
                      id='ddown5',
        ),
     
    
     html.Div(dcc.Graph(id='graph1')),
     html.Div(dcc.Graph(id='graph2')),
     html.Div(dcc.Graph(id='graph3')),
    
    
     dcc.Dropdown(options=[{'label': str(category), 'value': str(category)}  for category in df['category'].unique()],
                  id='ddown1',
    ),
     dcc.Dropdown(options=[{'label': str(tier), 'value': str(tier)}  for tier in df['tier'].unique()],
                  id='ddown2',
    ),
    
])
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1306, in dispatch
    cb = self.callback_map[output]
KeyError: 'my_h1.children'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv 

127.0.0.1 - - [29/Apr/2022 16:53:13] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [29/Apr/2022 16:53:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_1_0m1651242495.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_3_0m1648105790.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_3_0m1648105790.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_2m1648105790.min.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_3_0m1648105790.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2022 16:53:15] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_3_0m1648105790.min.js HTTP/1.1" 200 -
1

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1306, in dispatch
    cb = self.callback_map[output]
KeyError: 'my_h1.children'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv 

...

127.0.0.1 - - [29/Apr/2022 16:55:01] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1306, in dispatch
    cb = self.callback_map[output]
KeyError: 'my_h1.children'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv 

127.0.0.1 - - [29/Apr/2022 16:55:01] "POST /_dash-update-component HTTP/1.1" 500 -


Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/dash/dash.py", line 1306, in dispatch
    cb = self.callback_map[output]
KeyError: 'my_h1.children'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/home/ayah/anaconda3/lib/python3.9/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv 

127.0.0.1 - - [29/Apr/2022 16:55:03] "POST /_dash-update-component HTTP/1.1" 500 -
